In [65]:
import pandas as pd
import numpy as np

In [66]:
df = pd.read_parquet("../data/matches.parquet")

In [67]:
df.info()
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020 entries, 0 to 2019
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   round             2020 non-null   object        
 1   match_date        2016 non-null   datetime64[ns]
 2   home_team         2020 non-null   object        
 3   guest_team        2020 non-null   object        
 4   stadium           2020 non-null   object        
 5   date_added        2020 non-null   datetime64[ns]
 6   score_home_team   2016 non-null   Int64         
 7   score_guest_team  2016 non-null   Int64         
 8   winning_team      2016 non-null   object        
dtypes: Int64(2), datetime64[ns](2), object(5)
memory usage: 146.1+ KB
      round          match_date  home_team     guest_team         stadium  \
0  Rodada 1 2020-08-08 19:00:00  Fortaleza   Athletico-PR   Castelão (CE)   
1  Rodada 1 2020-08-08 19:30:00   Coritiba  Internacional   Couto Pereira  

In [68]:
df = df.dropna()

Historical performance: Rolling average of points per game for each team over last N matches (e.g., last 5-10 games)

In [69]:
import pandas as pd

# 1. Create a long-format performance DataFrame upfront
def create_performance_df(match_df):
    # Home team performances
    home = match_df[['match_date', 'home_team', 'score_home_team', 'score_guest_team', 'winning_team']].copy()
    home['team'] = home['home_team']
    home['opponent'] = home['score_guest_team']
    home['is_home'] = True
    home['points'] = home['winning_team'].map({'home': 3, 'draw': 1, 'guest': 0})
    
    # Away team performances
    away = match_df[['match_date', 'guest_team', 'score_home_team', 'score_guest_team', 'winning_team']].copy()
    away['team'] = away['guest_team']
    away['opponent'] = away['score_home_team']
    away['is_home'] = False
    away['points'] = away['winning_team'].map({'home': 0, 'draw': 1, 'guest': 3})
    
    # Combine and sort
    perf_df = pd.concat([home, away], ignore_index=True)
    perf_df = perf_df.sort_values(['team', 'match_date'])
    
    # Calculate match sequence number for each team
    perf_df['team_match_seq'] = perf_df.groupby('team').cumcount() + 1
    
    return perf_df[['match_date', 'team', 'opponent', 'is_home', 'points', 'team_match_seq']]

# 2. Create feature calculation functions
def calculate_features(perf_df, window=5):
    # Ensure chronological order
    perf_df = perf_df.sort_values(['team', 'match_date'])
    
    # Rolling features
    perf_df['rolling_points_avg'] = perf_df.groupby('team')['points'].transform(
        lambda x: x.rolling(window, min_periods=1).mean().shift(1))
    
    perf_df['rolling_points_std'] = perf_df.groupby('team')['points'].transform(
        lambda x: x.rolling(window, min_periods=1).std().shift(1))
    
    # Can add more features here (form, streaks, etc.)
    return perf_df

# 3. Create match-level features by joining performance data
def create_match_features(match_df, perf_df):
    # Home team features
    match_df = match_df.merge(
        perf_df.add_prefix('home_'),
        left_on=['match_date', 'home_team'],
        right_on=['home_match_date', 'home_team'],
        how='left'
    )
    
    # Away team features
    match_df = match_df.merge(
        perf_df.add_prefix('away_'),
        left_on=['match_date', 'guest_team'],
        right_on=['away_match_date', 'away_team'],
        how='left'
    )
    
    return match_df

# Main workflow
# Assuming df is your original DataFrame
perf_df = create_performance_df(df)
featured_perf_df = calculate_features(perf_df)
final_df = create_match_features(df, featured_perf_df)

Last 5 games win/draw/loss ratio for both teams

In [70]:
import pandas as pd
from collections import defaultdict

# Assuming final_df is your dataframe
# First sort by match date to ensure chronological order
final_df = final_df.sort_values('match_date')

# Create dictionaries to store team histories
team_history = defaultdict(list)

# Function to calculate last 5 matches performance
def get_last_5_performance(team, date):
    team_matches = team_history[team]
    last_5 = [m for m in team_matches if m['date'] < date][-5:]
    
    if not last_5:
        return {'win_ratio': None, 'draw_ratio': None, 'loss_ratio': None}
    
    wins = 0
    draws = 0
    losses = 0
    
    for match in last_5:
        if match['result'] == 'win':
            wins += 1
        elif match['result'] == 'draw':
            draws += 1
        else:
            losses += 1
    
    total = wins + draws + losses
    return {
        'win_ratio': wins / total,
        'draw_ratio': draws / total,
        'loss_ratio': losses / total
    }

# First pass to populate team histories
for _, row in final_df.iterrows():
    if pd.isna(row['match_date']) or pd.isna(row['winning_team']):
        continue
    
    home_team = row['home_team']
    guest_team = row['guest_team']
    match_date = row['match_date']
    
    # Determine results for each team
    if row['winning_team'] == 'home':
        home_result = 'win'
        guest_result = 'loss'
    elif row['winning_team'] == 'guest':
        home_result = 'loss'
        guest_result = 'win'
    else:  # draw
        home_result = 'draw'
        guest_result = 'draw'
    
    # Add to team histories
    team_history[home_team].append({
        'date': match_date,
        'result': home_result,
        'opponent': guest_team,
        'home_away': 'home'
    })
    
    team_history[guest_team].append({
        'date': match_date,
        'result': guest_result,
        'opponent': home_team,
        'home_away': 'away'
    })

# Second pass to calculate last 5 performance
home_win_ratios = []
home_draw_ratios = []
home_loss_ratios = []
guest_win_ratios = []
guest_draw_ratios = []
guest_loss_ratios = []

for _, row in final_df.iterrows():
    if pd.isna(row['match_date']):
        home_win_ratios.append(None)
        home_draw_ratios.append(None)
        home_loss_ratios.append(None)
        guest_win_ratios.append(None)
        guest_draw_ratios.append(None)
        guest_loss_ratios.append(None)
        continue
    
    home_perf = get_last_5_performance(row['home_team'], row['match_date'])
    guest_perf = get_last_5_performance(row['guest_team'], row['match_date'])
    
    home_win_ratios.append(home_perf['win_ratio'])
    home_draw_ratios.append(home_perf['draw_ratio'])
    home_loss_ratios.append(home_perf['loss_ratio'])
    
    guest_win_ratios.append(guest_perf['win_ratio'])
    guest_draw_ratios.append(guest_perf['draw_ratio'])
    guest_loss_ratios.append(guest_perf['loss_ratio'])

# Add new columns to dataframe
final_df['home_win_ratio_last5'] = home_win_ratios
final_df['home_draw_ratio_last5'] = home_draw_ratios
final_df['home_loss_ratio_last5'] = home_loss_ratios
final_df['guest_win_ratio_last5'] = guest_win_ratios
final_df['guest_draw_ratio_last5'] = guest_draw_ratios
final_df['guest_loss_ratio_last5'] = guest_loss_ratios

Goals scored/conceded in last N matches

In [71]:
# Assuming your dataframe is named 'df'
# First, sort by match_date to ensure chronological order
final_df = final_df.sort_values('match_date').reset_index(drop=True)

# Define the number of previous matches to consider
N = 5  # You can adjust this based on your needs

# Create a long format dataframe with all team performances
home_games = final_df[['match_date', 'home_team', 'score_home_team', 'score_guest_team']].copy()
home_games['is_home'] = True
home_games = home_games.rename(columns={
    'home_team': 'team',
    'score_home_team': 'goals_scored',
    'score_guest_team': 'goals_conceded'
})

away_games = final_df[['match_date', 'guest_team', 'score_guest_team', 'score_home_team']].copy()
away_games['is_home'] = False
away_games = away_games.rename(columns={
    'guest_team': 'team',
    'score_guest_team': 'goals_scored',
    'score_home_team': 'goals_conceded'
})

all_games = pd.concat([home_games, away_games]).sort_values(['team', 'match_date'])

# Calculate rolling averages
for team in all_games['team'].unique():
    team_mask = all_games['team'] == team
    all_games.loc[team_mask, 'goals_scored_avg'] = (
        all_games.loc[team_mask, 'goals_scored']
        .rolling(N, min_periods=1)
        .mean()
        .shift(1)  # Use previous matches only
    )
    all_games.loc[team_mask, 'goals_conceded_avg'] = (
        all_games.loc[team_mask, 'goals_conceded']
        .rolling(N, min_periods=1)
        .mean()
        .shift(1)
    )

# Merge back to original dataframe
# For home teams
final_df = final_df.merge(
    all_games[all_games['is_home']][['match_date', 'team', 'goals_scored_avg', 'goals_conceded_avg']],
    left_on=['match_date', 'home_team'],
    right_on=['match_date', 'team'],
    how='left'
).rename(columns={
    'goals_scored_avg': f'home_team_goals_scored_last_{N}',
    'goals_conceded_avg': f'home_team_goals_conceded_last_{N}'
}).drop(columns=['team'])

# For away teams
final_df = final_df.merge(
    all_games[~all_games['is_home']][['match_date', 'team', 'goals_scored_avg', 'goals_conceded_avg']],
    left_on=['match_date', 'guest_team'],
    right_on=['match_date', 'team'],
    how='left'
).rename(columns={
    'goals_scored_avg': f'guest_team_goals_scored_last_{N}',
    'goals_conceded_avg': f'guest_team_goals_conceded_last_{N}'
}).drop(columns=['team'])

In [72]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2016 entries, 0 to 2015
Data columns (total 34 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   round                             2016 non-null   object        
 1   match_date                        2016 non-null   datetime64[ns]
 2   home_team                         2016 non-null   object        
 3   guest_team                        2016 non-null   object        
 4   stadium                           2016 non-null   object        
 5   date_added                        2016 non-null   datetime64[ns]
 6   score_home_team                   2016 non-null   Int64         
 7   score_guest_team                  2016 non-null   Int64         
 8   winning_team                      2016 non-null   object        
 9   home_match_date                   2016 non-null   datetime64[ns]
 10  home_opponent                     2016 non-null 

In [73]:
# filter flamengo in home or guest team
final_df[(final_df['home_team'] == 'Flamengo') | (final_df['guest_team'] == 'Flamengo')][['home_team','guest_team', 'score_home_team', 'score_guest_team', 'home_team_goals_scored_last_5', 
          'home_team_goals_conceded_last_5', 'guest_team_goals_scored_last_5', 'guest_team_goals_conceded_last_5', ]].head(10)

,home_team,guest_team,score_home_team,score_guest_team,home_team_goals_scored_last_5,home_team_goals_conceded_last_5,guest_team_goals_scored_last_5,guest_team_goals_conceded_last_5
3,Flamengo,Atlético-MG,0,1,NaN,NaN,NaN,NaN
10,Atlético-GO,Flamengo,3,0,NaN,NaN,0.000000,1.000000
17,Coritiba,Flamengo,0,1,0.000000,1.000000,0.000000,2.000000
26,Flamengo,Grêmio,1,1,0.333333,1.333333,0.666667,0.333333
39,Flamengo,Botafogo,1,1,0.500000,1.250000,1.000000,0.666667
53,Santos,Flamengo,0,1,1.200000,1.200000,0.600000,1.200000
59,Bahia,Flamengo,3,5,1.000000,1.000000,0.800000,1.000000
65,Flamengo,Fortaleza,2,1,1.800000,1.000000,1.400000,0.600000
79,Fluminense,Flamengo,1,2,1.200000,1.400000,2.000000,1.200000
91,Ceará,Flamengo,2,0,1.000000,0.600000,2.200000,1.200000


In [74]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Load data (assuming `final_df` from previous steps)
df = final_df.copy()

# Encode target variable (winning_team: home/draw/guest)
le = LabelEncoder()
df['target'] = le.fit_transform(df['winning_team'])  # 0=home, 1=draw, 2=guest

# Select features (modify as needed)
features = [
    'home_rolling_points_avg', 'away_rolling_points_avg',

    'home_win_ratio_last5', 'home_draw_ratio_last5', 'home_loss_ratio_last5', 'guest_win_ratio_last5', 'guest_draw_ratio_last5', 'guest_loss_ratio_last5',

    'home_team_goals_scored_last_5', 'home_team_goals_conceded_last_5', 'guest_team_goals_scored_last_5', 'guest_team_goals_conceded_last_5'
]

X = df[features]
y = df['target']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Calculate class weights (inverse of class frequencies)
class_weights = len(y_train) / (3 * np.bincount(y_train))
sample_weights = np.array([class_weights[y] for y in y_train])

model = XGBClassifier(
    objective='multi:softmax',
    num_class=3,
    n_estimators=200,
    max_depth=4,
    learning_rate=0.05,
    random_state=42
)

# Pass sample_weight during training
model.fit(X_train, y_train, sample_weight=sample_weights)

# Predictions
y_pred = model.predict(X_test)

# Decode labels back to original values
class_names = le.inverse_transform([0, 1, 2])  # ['home', 'draw', 'guest']

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)

print(cm)
# # Display the confusion matrix
# disp = ConfusionMatrixDisplay(
#     confusion_matrix=cm,
#     display_labels=class_names  # ['home', 'draw', 'guest']
# )
# disp.plot(cmap=plt.cm.Blues)
# plt.title('Confusion Matrix - Match Outcome Prediction')
# plt.show()

print(classification_report(
    y_test,
    y_pred,
    target_names=class_names,
    digits=3
))

# Get feature importances
importance = model.feature_importances_
feature_importance = pd.DataFrame({
    'Feature': features,
    'Importance': importance
}).sort_values('Importance', ascending=False)

print(feature_importance)

[[19 43 41]
 [27 49 29]
 [52 60 84]]
              precision    recall  f1-score   support

        draw      0.194     0.184     0.189       103
       guest      0.322     0.467     0.381       105
        home      0.545     0.429     0.480       196

    accuracy                          0.376       404
   macro avg      0.354     0.360     0.350       404
weighted avg      0.398     0.376     0.380       404

                             Feature  Importance
2               home_win_ratio_last5    0.106819
8      home_team_goals_scored_last_5    0.093277
1            away_rolling_points_avg    0.088564
7             guest_loss_ratio_last5    0.086887
0            home_rolling_points_avg    0.084064
4              home_loss_ratio_last5    0.082987
5              guest_win_ratio_last5    0.081497
11  guest_team_goals_conceded_last_5    0.079790
6             guest_draw_ratio_last5    0.076631
10    guest_team_goals_scored_last_5    0.074863
9    home_team_goals_conceded_last_5    0.0